In [1]:
import scanpy as sc
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from tqdm import tqdm   

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class VAEEverything(nn.Module):
    def __init__(self, input_dim,
                 alpha,
                 enc_hidden_layer_dim,
                 enc_nbr_hidden_layers,
                 rec_hidden_layer_dim,
                 rec_nbr_hidden_layers,
                 pert_pred_hidden_layer_dim,
                 pert_pred_nbr_hidden_layers,
                 latent_dim,
                 gene_embedding_dim):
        super().__init__()
        
        self.input_dim = input_dim
        self.alpha = alpha
        self.fc1_enc = nn.Linear(input_dim, enc_hidden_layer_dim)
        self.relu1_enc = nn.LeakyReLU()
        self.fc2_enc = nn.Linear(enc_hidden_layer_dim, enc_hidden_layer_dim)
        self.relu2_enc = nn.LeakyReLU()
        self.fc3_enc = nn.Linear(enc_hidden_layer_dim, enc_hidden_layer_dim)
        self.relu3_enc = nn.LeakyReLU()
        self.fc4_enc = nn.Linear(enc_hidden_layer_dim, enc_hidden_layer_dim)
        self.relu4_enc = nn.LeakyReLU()
        #self.fc5_enc = nn.Linear(enc_hidden_layer_dim, enc_hidden_layer_dim)
        #self.relu5_enc = nn.LeakyReLU()

        self.encoder = nn.Sequential(*[self.fc1_enc, self.relu1_enc, self.fc2_enc, self.relu2_enc, self.fc3_enc, self.relu3_enc, self.fc4_enc, self.relu4_enc]) #, self.fc5_enc, self.relu5_enc])


        self.fc_mu = nn.Linear(enc_hidden_layer_dim, latent_dim)
        self.fc_var = nn.Linear(enc_hidden_layer_dim, latent_dim)
        
        # Learnable gene embeddings
        self.gene_embeddings = nn.Embedding(input_dim, gene_embedding_dim)
        self.gene_embeddings.freeze = False\

        self.fc1_dec_rec = nn.Linear(latent_dim + gene_embedding_dim, rec_hidden_layer_dim)
        self.relu1_dec_rec = nn.LeakyReLU()
        self.fc2_dec_rec = nn.Linear(rec_hidden_layer_dim, rec_hidden_layer_dim)
        self.relu2_dec_rec = nn.LeakyReLU()
        self.fc3_dec_rec = nn.Linear(rec_hidden_layer_dim, rec_hidden_layer_dim)
        self.relu3_dec_rec = nn.LeakyReLU()
        self.fc4_dec_rec = nn.Linear(rec_hidden_layer_dim, rec_hidden_layer_dim)
        self.relu4_dec_rec = nn.LeakyReLU()

        self.fc_raw_counts = nn.Linear(rec_hidden_layer_dim, 1)

        
        self.fc_sparsity = nn.Linear(rec_hidden_layer_dim, 1)
        self.sigmoid_sparsity = nn.Sigmoid()

        self.raw_counts_decoder = nn.Sequential(*[self.fc1_dec_rec, self.relu1_dec_rec, self.fc2_dec_rec, 
                                                  self.relu2_dec_rec, self.fc3_dec_rec, self.relu3_dec_rec, 
                                                  self.fc4_dec_rec, self.relu4_dec_rec, self.fc_raw_counts])
        

        self.sparsity_decoder = nn.Sequential(*[self.fc1_dec_rec, self.relu1_dec_rec, self.fc2_dec_rec,
                                                self.relu2_dec_rec, self.fc3_dec_rec, self.relu3_dec_rec,
                                                self.fc4_dec_rec, self.relu4_dec_rec, self.fc_sparsity, self.sigmoid_sparsity])
        
        
      

        
        
        # Build Perturbation Decoder layers dynamically

        self.fc1_dec_pert = nn.Linear(latent_dim + gene_embedding_dim, pert_pred_hidden_layer_dim)
        self.relu1_dec_pert = nn.LeakyReLU()
        self.fc2_dec_pert = nn.Linear(pert_pred_hidden_layer_dim, pert_pred_hidden_layer_dim)
        self.relu2_dec_pert = nn.LeakyReLU()
        self.fc3_dec_pert = nn.Linear(pert_pred_hidden_layer_dim, pert_pred_hidden_layer_dim)
        self.relu3_dec_pert = nn.LeakyReLU()
        self.fc4_dec_pert = nn.Linear(pert_pred_hidden_layer_dim, pert_pred_hidden_layer_dim)
        self.relu4_dec_pert = nn.LeakyReLU()
        self.fc5_dec_pert = nn.Linear(pert_pred_hidden_layer_dim, input_dim)


        self.perturbation_decoder = nn.Sequential(*[self.fc1_dec_pert, self.relu1_dec_pert, self.fc2_dec_pert,
                                                    self.relu2_dec_pert, self.fc3_dec_pert, self.relu3_dec_pert,
                                                    self.fc4_dec_pert, self.relu4_dec_pert, self.fc5_dec_pert])


    def encode(self, x):
        """
        Forward pass for the encoder
        
        Args:
        x: input tensor of shape (batch_size, input_dim)
        
        Returns:
        mu: mean of the latent variable of shape (batch_size, latent_dim)
        """
        x = self.encoder(x)
        mu = self.fc_mu(x)
        log_var = self.fc_var(x)
        return mu, log_var

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std


    def reconstruction_decode(self, z):
        """
        Forward pass for the reconstruction mode

        Args:
        z: latent variable of shape (batch_size, latent_dim)
        
        Returns:
        expression_value: predicted expression value of shape (batch_size, num_genes)
        sparsity_value: predicted sparsity value of shape (batch_size, num_genes)
        """

        z_replicated = z.unsqueeze(1).expand(-1, self.gene_embeddings.weight.size(0), -1)

        reshaped_gene_emb = self.gene_embeddings.weight.unsqueeze(0).expand(z.size(0), -1, -1)



        
        z_and_gene = torch.cat((z_replicated, reshaped_gene_emb), axis=-1)

        expression_value = self.raw_counts_decoder(z_and_gene).squeeze(-1)

        sparsity_value = self.sparsity_decoder(z_and_gene).squeeze(-1)

        return expression_value, sparsity_value



    def perturbation_decode(self, z, gene_idx):

        """
        Args:
        z: latent variable of shape (batch_size, latent_dim)
        gene_idx: index of the gene to perturb

        Returns:

        """
        # Get gene embedding for the specific gene
        gene_emb = self.gene_embeddings[gene_idx]


        batch_size = z.size(0)
        gene_emb = gene_emb.unsqueeze(0).expand(batch_size, -1)


        # Concatenate cell embedding with gene embedding
        decoder_input = torch.cat([z, gene_emb], dim=-1)
        
        # Get perturbation output
        perturbed_value = self.perturbation_decoder(decoder_input)
        return perturbed_value.squeeze()

    def forward(self, x, gene_idx = None, mode='reconstruction'):
        # Encode
        mu, log_var = self.encode(x)
        
        # Reparameterize
        z = self.reparameterize(mu, log_var)
        
        # Decode based on mode
        if mode == 'reconstruction':
            expression_value, is_expressed = self.reconstruction_decode(z)
            return expression_value, is_expressed, mu, log_var
        else:  # perturbation mode

            perturbed_value = self.perturbation_decode(z, gene_idx)
            return perturbed_value, mu, log_var
        
    
    def forward_reconstruction(self, x):
        return self.forward(x, mode='reconstruction')
    
    def forward_perturbation(self, x, gene_idx):
        return self.forward(x, gene_idx, mode='perturbation')


        
    def rec_loss_function(self, recon_x, x, mu, logvar):


        recon_loss = F.mse_loss(recon_x, x.view(-1, self.input_dim), reduction='sum')
        KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        return recon_loss + self.alpha * KLD



    def pert_loss_function(self, perturbed_x, x):
        pert_loss = F.mse_loss(perturbed_x, x.view(-1, self.input_dim), reduction='sum')
        #KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        return pert_loss





In [3]:
def get_train_eval_idxs(
    adata, control_pert, holdout_cells, holdout_perts, 
    cell_col='cell_type', pert_col='pert_type', verbose=2
):
    # here we set up the train/eval and control/pert sets
    # set the idx of the controls
    control_idx = adata.obs[pert_col] == control_pert
    # set the idx of the perts (currently just "all not control")
    pert_idx = adata.obs[pert_col] != control_pert
    # set the hold out cell-type/pert
    eval_idx = control_idx & False
    for holdout_cell, holdout_pert in zip(holdout_cells, holdout_perts):
        eval_idx |= (adata.obs[cell_col] == holdout_cell) & (adata.obs[pert_col] == holdout_pert)
    eval_cell_idx = adata.obs[cell_col].isin(holdout_cells)
    eval_pert_idx = adata.obs[pert_col].isin(holdout_perts)
    eval_idx = eval_cell_idx & eval_pert_idx
    if verbose > 0:
        print(f"Controls: {(control_idx & ~eval_idx).sum()}, Perturbations: {(pert_idx & ~eval_idx).sum()},  Eval: {eval_idx.sum()}")
    return control_idx, pert_idx, eval_idx, eval_cell_idx, eval_pert_idx

def get_identity_features(adata, cell_col='cell_type', pert_col='perturb', cell_type_features=True):
    perts = pd.get_dummies(adata.obs[pert_col]).values.astype(float)
    cell_types = pd.get_dummies(adata.obs[cell_col]).values
    if cell_type_features:
        combo = pd.get_dummies(adata.obs[cell_col].astype(str) + adata.obs[pert_col].astype(str)).values
        idx = (combo!=0).argmax(axis=0)
        pert_mat = np.hstack([cell_types, perts])[idx, :].astype('float32')
    else:
        combo = perts
        idx = (combo!=0).argmax(axis=0)
        pert_mat = perts[idx, :].astype('float32')
    
    pert_ids = combo.argmax(axis=1)
    cell_types = cell_types.argmax(axis=1)
    return pert_ids, pert_mat, cell_types

def get_train_eval(
    X, pert_ids, cell_types, control_idx, pert_idx, eval_idx, eval_cell_idx, eval_pert_idx
):
    # set train and eval split
    control_train = X[control_idx & ~eval_idx]
    pert_train = X[pert_idx & ~eval_idx]
    pert_ids_train =  pert_ids[pert_idx & ~eval_idx]
    control_cell_types = cell_types[control_idx & ~eval_idx]
    pert_cell_types = cell_types[pert_idx & ~eval_idx]

    control_eval = X[control_idx & eval_cell_idx]
    pert_eval = X[eval_idx]
    pert_ids_eval = pert_ids[eval_idx]
    return control_train, pert_train, pert_ids_train, control_cell_types, pert_cell_types, control_eval, pert_eval, pert_ids_eval


class SCFMDataset(torch.utils.data.Dataset):
    def __init__(
        self, source, target, pert_ids, pert_mat, source_strata, target_strata, 
        probs=None, size=int(1e4), batch_size=32
    ):
        source, target = np.array(source), np.array(target)
        pert_ids, pert_mat = np.array(pert_ids), np.array(pert_mat)
        
        assert target.shape[0] == pert_ids.shape[0]
        assert source.shape[0] == source_strata.shape[0]
        assert target.shape[0] == target_strata.shape[0]
        
        self.size = size
        self.source_strata = source_strata
        self.target_strata = target_strata
        self.strata = np.unique(source_strata)
        self.num_strata = len(self.strata)
        
        self.pert_ids = np.unique(pert_ids)
        
        self.source = [source[source_strata == stratum] for stratum in self.strata]
        self.target = [
            [
                target[target_strata == stratum][pert_ids[target_strata == stratum] == pert_id] 
                for pert_id in self.pert_ids
            ] for stratum in self.strata
        ]
        self.pert_ids = [
            [
                pert_ids[target_strata == stratum][pert_ids[target_strata == stratum] == pert_id] 
                for pert_id in self.pert_ids
            ] for stratum in self.strata
        ]
        self.pert_mat = pert_mat
        

    def __len__(self):
        return self.size

    def __getitem__(self, strata_idx):
        stratum, idx = strata_idx
        sidx = np.random.choice(self.source[stratum[0]].shape[0])
        return (
            self.source[stratum[0]][sidx],
            self.target[stratum[0]][stratum[1]][idx],
            self.pert_mat[self.pert_ids[stratum[0]][stratum[1]][idx]],
        )

from torch.utils.data import BatchSampler, SequentialSampler, Sampler
from typing import Iterator, Iterable, Optional, Sequence, List, TypeVar, Generic, Sized, Union

class StratifiedBatchSampler(Sampler[List[int]]):
    def __init__(
        self, ns, batch_size: int
    ) -> None:
        # Since collections.abc.Iterable does not check for `__getitem__`, which
        # is one way for an object to be an iterable, we don't do an `isinstance`
        # check here.
        if not isinstance(batch_size, int) or isinstance(batch_size, bool) or \
                batch_size <= 0:
            raise ValueError(f"batch_size should be a positive integer value, but got batch_size={batch_size}")
        
        self.num_strata = np.prod(ns.shape)
        self.ns = ns
        self.probs = ns.flatten() / np.sum(ns)
        print("Strata probs", np.sort(self.probs))
        self.batch_size = batch_size
        self.batch_sizes = np.minimum(ns, batch_size)
    
    def get_random_stratum(self):
        linear_idx = np.random.choice(self.num_strata, p=self.probs)
        stratum = np.unravel_index(linear_idx, self.ns.shape)
        return stratum

    def __iter__(self) -> Iterator[List[int]]:
        # Implemented based on the benchmarking in https://github.com/pytorch/pytorch/pull/76951
        while True:
            stratum = self.get_random_stratum()
            try:
                batch_stratum = np.repeat(np.array(stratum)[None, :], self.batch_sizes[stratum], axis=0)
                batch = np.random.choice(self.ns[stratum], self.batch_sizes[stratum], replace=False)
                yield zip(batch_stratum, batch)
            except StopIteration:
                break

    def __len__(self) -> int:
        # Can only be called if self.sampler has __len__ implemented
        # We cannot enforce this condition, so we turn off typechecking for the
        # implementation below.
        # Somewhat related: see NOTE [ Lack of Default `__len__` in Python Abstract Base Classes ]
        return np.sum(self.ns) // self.batch_size

from torchcfm.conditional_flow_matching import (
    ExactOptimalTransportConditionalFlowMatcher,OTPlanSampler
)
def ot_collate(
    batch,
    return_noise=False,
    ot_sampler = OTPlanSampler(method="exact")
):

    batch = list(zip(*batch))
    batch = [torch.tensor(np.array(x)) for x in batch]
    control, target = batch[:2]
    
    x0, x1 = ot_sampler.sample_plan(control, target)

    return x0, x1, *batch[2:]


In [3]:
with open("/om/group/abugoot/Projects/Omnicell_datasets/satija_IFNB_raw/Seurat_IFNB.h5ad", "rb") as f:
    adata = sc.read_h5ad(f)




In [4]:
sc.pp.normalize_total(adata, target_sum=10000)
sc.pp.log1p(adata)


In [12]:
CELL_TYPE_KEY = 'cell_type'
PERT_KEY = 'gene'
CONTROL_PERT = 'NT'

PERT_INDEX_KEY = "pert_type"
adata.var['gene']



1            TSPAN6
2              TNMD
3              DPM1
4             SCYL3
5          C1orf112
            ...    
34021    AP006545.2
34022    AL359198.1
34023    AL360219.1
34024    AL138966.2
34025    AC092634.8
Name: gene, Length: 34025, dtype: object

In [13]:
gene_map = {k: i for i, k in enumerate(adata.var['gene'])}
gene_map = gene_map | {'NT': max(gene_map.values()) + 1}
gene_unmap = {gene_map[k]: k for k in gene_map}
perts = adata.obs.gene.unique().map(gene_map)
adata.obs['pert_type'] = adata.obs.gene.map(gene_map)

adata.obs

,orig.ident,nCount_RNA,nFeature_RNA,sample,bc1_well,bc2_well,bc3_well,percent.mito,cell_type,pathway,RNA_snn_res.0.9,seurat_clusters,sample_ID,Batch_info,guide,gene,mixscale_score,pert_type
07_48_88_1_1_1_1_1_1_1_1_1,7,9816,4122,A549_IFNB,A7,D12,H4,1.161369,A549,IFNB,15.0,15.0,sample_1,Rep1,TRAFD1g3,TRAFD1,-0.290358,6608
06_04_63_1_1_1_1_1_1_1_1_1,6,9359,4112,A549_IFNB,A6,A4,F3,3.835880,A549,IFNB,15.0,15.0,sample_1,Rep1,HES4g3,HES4,0.121449,14836
06_28_67_1_1_1_1_1_1_1_1_1,6,8999,3854,A549_IFNB,A6,C4,F7,9.189910,A549,IFNB,15.0,15.0,sample_1,Rep1,NTg8,NT,0.000000,34025
06_27_93_1_1_1_1_1_1_1_1_1,6,8384,3600,A549_IFNB,A6,C3,H9,3.268130,A549,IFNB,15.0,15.0,sample_1,Rep1,STAT5Ag1,STAT5A,0.377627,5498
06_81_38_1_1_1_1_1_1_1_1_1,6,7925,3580,A549_IFNB,A6,G9,D2,3.798107,A549,IFNB,15.0,15.0,sample_1,Rep1,STAT4g2,STAT4,1.000000,7199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
09_10_48_2_2,9,2362,1491,MCF7_IFNB,A9,A10,D12,1.820491,MCF7,IFNB,NaN,NaN,sample_16,Rep2,NFE2L3g3,NFE2L3,-0.009685,655
10_73_27_2_2,10,2143,1483,MCF7_IFNB,A10,G1,C3,0.326645,MCF7,IFNB,NaN,NaN,sample_16,Rep2,IRF3g2,IRF3,-0.402627,5488
10_24_48_2_2,10,1921,1322,MCF7_IFNB,A10,B12,D12,6.507028,MCF7,IFNB,NaN,NaN,sample_16,Rep2,STAT6g2,STAT6,-0.356580,11158
10_12_27_2_2,10,1676,1169,MCF7_IFNB,A10,A12,C3,2.923628,MCF7,IFNB,NaN,NaN,sample_16,Rep2,HERC5g3,HERC5,-0.045158,7245


In [9]:

pert_ids = np.array(adata.obs['pert_type'])
pert_mat = np.arange(pert_ids.max() + 1)[:, None]

control_pert, holdout_cells, holdout_perts = gene_map['NT'], ['HT29'], [gene_map['USP18']]

In [10]:
control_idx, pert_idx, eval_idx, eval_cell_idx, eval_pert_idx = get_train_eval_idxs(
    adata, control_pert, holdout_cells, holdout_perts, cell_col=CELL_TYPE_KEY, pert_col=PERT_INDEX_KEY
)

_, _, cell_types = get_identity_features(
    adata, cell_col=CELL_TYPE_KEY, pert_col=PERT_INDEX_KEY, cell_type_features=False
)

X = adata.X.toarray()

control_train, pert_train, pert_ids_train, control_cell_types, pert_cell_types, control_eval, pert_eval, pert_ids_eval = get_train_eval(
    X, pert_ids, cell_types, control_idx, pert_idx, eval_idx, eval_cell_idx, eval_pert_idx
)




Controls: 14582, Perturbations: 313080,  Eval: 880


In [37]:
del dataloader_rec

In [11]:
batch_size = 32


In [12]:



train = np.vstack([control_train, pert_train])



# Create a custom Dataset class
class NumpyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, index):
        return torch.from_numpy(self.data[index]).float()

    def __len__(self):
        return len(self.data)

# Create an instance of the dataset
dataset_rec = NumpyDataset(train)

del train


In [13]:

dataloader_rec = DataLoader(dataset_rec, batch_size=batch_size, shuffle=True)


In [109]:
dset = SCFMDataset(
    control_train, pert_train, 
    pert_ids_train, pert_mat, 
    control_cell_types, pert_cell_types,
    batch_size=batch_size, size=X.shape[0]
)
ns = np.array([[t.shape[0] for t in ts] for ts in dset.target])
dataloader_pert = torch.utils.data.DataLoader(
    dset, collate_fn=ot_collate, 
    batch_sampler=StratifiedBatchSampler(
        ns=ns, batch_size=batch_size
    )
)

: 

: 

: 

In [14]:
device = 'cuda'
# device = 'cpu'

dropout_rate = 0.75
learning_rate = 0.1
alpha =  0.01
epochs =  100


In [15]:
0.1
alpha =  0.01
epochs =  100


In [116]:

INPUT_DIM = adata.shape[1]
ENC_HIDDEN_LAYER_DIM = 500
ENC_NBR_HIDDEN_LAYERS = 4
REC_HIDDEN_LAYER_DIM = 500
REC_NBR_HIDDEN_LAYERS = 4
PERT_PRED_HIDDEN_LAYER_DIM = 500
PERT_PRED_NBR_HIDDEN_LAYERS = 4
LATENT_DIM = 64
GENE_EMBEDDING_DIM = 8


model = VAEEverything(
    input_dim=INPUT_DIM,
    alpha=alpha,
    enc_hidden_layer_dim=ENC_HIDDEN_LAYER_DIM,
    enc_nbr_hidden_layers=ENC_NBR_HIDDEN_LAYERS,
    rec_hidden_layer_dim=REC_HIDDEN_LAYER_DIM,
    rec_nbr_hidden_layers=REC_NBR_HIDDEN_LAYERS,
    pert_pred_hidden_layer_dim=PERT_PRED_HIDDEN_LAYER_DIM,
    pert_pred_nbr_hidden_layers=PERT_PRED_NBR_HIDDEN_LAYERS,
    latent_dim=LATENT_DIM,
    gene_embedding_dim=GENE_EMBEDDING_DIM).to(device)

In [117]:
import math
base_learning_rate = 2e-4
weight_decay=0.0
total_epoch = 1000
warmup_epoch = 5
lr_step = 32
lr_func = lambda epoch: min((epoch + 1) / (warmup_epoch + 1e-5), 0.5 * (math.cos(epoch / total_epoch * math.pi) + 1))


optim = torch.optim.Adam(model.parameters(), lr=base_learning_rate, weight_decay=weight_decay)
#lr_scheduler = torch.optim.lr_scheduler.LambdaLR(optim, lr_lambda=lr_func, verbose=True)



In [114]:
del optim
del model
del lr_scheduler

NameError: name 'lr_scheduler' is not defined

In [115]:
del logvar
del loss
del recon
del is_expressed
del batch
del idx
del mu



NameError: name 'idx' is not defined

In [104]:
optim.zero_grad()


NameError: name 'optim' is not defined

In [ ]:

import gc
torch.cuda.empty_cache()
gc.collect()


1897

In [97]:
IFNAR1_idx = torch.Tensor([gene_map['IFNAR1']]).long().to(device) 
IFNAR2_idx = torch.Tensor([gene_map['IFNAR2']]).long().to(device)
number_rand_genes = 30



In [98]:

print(f"IFNAR1: {IFNAR1_idx}, IFNAR2: {IFNAR2_idx}")

IFNAR1: tensor([7723], device='cuda:0'), IFNAR2: tensor([9659], device='cuda:0')


In [ ]:

step_count = 0
for e in range(10):

    model.train()
    losses = {'loss': [], 'control': [], "IFNAR1_to_random": [], "IFNAR1_IFNAR2": [], "mean_rec": [], "mean_batch": []}
    for batch in (pbar := tqdm(iter(dataloader_rec))):
        step_count += 1


        optim.zero_grad()
        batch = batch.to(device)



        sparsity_mask = (batch > 0).float()

        recon, is_expressed, mu, logvar = model.forward(batch, mode='reconstruction')

        diff = (batch - recon) ** 4


        sparsity_loss = torch.sum((is_expressed - sparsity_mask)**2)

        raw_counts_loss_sparsified = torch.sum(diff * sparsity_mask)
        raw_counts_loss = torch.sum(diff)
        KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

        loss = raw_counts_loss_sparsified + alpha * KLD + sparsity_loss
        #loss = raw_counts_loss + alpha * KLD + sparsity_loss
        loss.backward()
        optim.step()

 

        with torch.no_grad():
            random_indices = torch.randint(0, INPUT_DIM, (number_rand_genes,)).to(device)

            IFNAR1_emb = model.gene_embeddings(IFNAR1_idx)
            IFNAR2_emb = model.gene_embeddings(IFNAR2_idx)

            random_genes = model.gene_embeddings(random_indices)

            dist_IFNAR1_to_random = torch.norm(IFNAR1_emb - random_genes, p=2, dim=-1)

            avg_dist_IFNAR1_to_random = torch.mean(dist_IFNAR1_to_random)



            embedding_dist = torch.norm(IFNAR1_emb - IFNAR2_emb, p=2)

      
        mean_rec = torch.mean(recon)
        mean_batch = torch.mean(batch)

        losses["mean_rec"].append(mean_rec.item())
        losses["mean_batch"].append(mean_batch.item())
        losses['loss'].append(loss.item())
        losses['IFNAR1_to_random'].append(avg_dist_IFNAR1_to_random.item())
        losses['IFNAR1_IFNAR2'].append(embedding_dist.item())
        
        pbar.set_description(
            f"""loss: {np.array(losses['loss'])[-lr_step:].mean():.3f}, IFNA1_to_random: {np.array(losses['IFNAR1_to_random'])[-lr_step:].mean():.3f}, IFNAR1_IFNAR2: {np.array(losses['IFNAR1_IFNAR2'])[-lr_step:].mean():.3f}"""
        )
        if(step_count % lr_step == 0):
             print(f"mean_rec: {np.array(losses['mean_rec'])[-lr_step:].mean():.3f}, mean_batch: {np.array(losses['mean_batch'])[-lr_step:].mean():.3f}")


    torch.save(model, "model.pt")


loss: 333787.277, IFNA1_to_random: 3.211, IFNAR1_IFNAR2: 2.309:   0%| | 32/10240 [00:24<2:12:2

mean_rec: 1.118, mean_batch: 0.109


loss: 153346.600, IFNA1_to_random: 3.201, IFNAR1_IFNAR2: 2.308:   1%| | 64/10240 [00:49<2:12:4

mean_rec: 1.354, mean_batch: 0.108


loss: 144170.031, IFNA1_to_random: 3.208, IFNAR1_IFNAR2: 2.305:   1%| | 96/10240 [01:15<2:12:1

...

loss: 53688.919, IFNA1_to_random: 3.827, IFNAR1_IFNAR2: 2.194:  38%|▍| 3840/10240 [50:19<1:23:

mean_rec: 1.283, mean_batch: 0.109


loss: 53788.234, IFNA1_to_random: 3.835, IFNAR1_IFNAR2: 2.197:  38%|▍| 3872/10240 [50:44<1:23:

mean_rec: 1.280, mean_batch: 0.109


loss: 53996.375, IFNA1_to_random: 3.826, IFNAR1_IFNAR2: 2.197:  38%|▍| 3874/10240 [50:45<1:23:

In [5]:
model = torch.load("notebooks/model.pt")

/tmp/ipykernel_3730038/37884352.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("notebooks/model.pt")


In [8]:
model
device = 'cuda'

In [9]:
model.to(device)
model.eval()

VAEEverything(
  (fc1_enc): Linear(in_features=34025, out_features=500, bias=True)
  (relu1_enc): LeakyReLU(negative_slope=0.01)
  (fc2_enc): Linear(in_features=500, out_features=500, bias=True)
  (relu2_enc): LeakyReLU(negative_slope=0.01)
  (fc3_enc): Linear(in_features=500, out_features=500, bias=True)
  (relu3_enc): LeakyReLU(negative_slope=0.01)
  (fc4_enc): Linear(in_features=500, out_features=500, bias=True)
  (relu4_enc): LeakyReLU(negative_slope=0.01)
  (encoder): Sequential(
    (0): Linear(in_features=34025, out_features=500, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=500, out_features=500, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=500, out_features=500, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=500, out_features=500, bias=True)
    (7): LeakyReLU(negative_slope=0.01)
  )
  (fc_mu): Linear(in_features=500, out_features=64, bias=True)
  (fc_var): Linear(in_features=50

In [10]:
data_np = adata.X.toarray()

In [ ]:
adata.X = data_np


In [16]:
adata.obs[CELL_TYPE_KEY].unique()


['A549', 'BXPC3', 'HAP1', 'HT29', 'K562', 'MCF7']
Categories (6, object): ['A549', 'BXPC3', 'HAP1', 'HT29', 'K562', 'MCF7']

In [ ]:
control_adata = adata[(adata.obs[PERT_KEY] == CONTROL_PERT) & (adata.obs[CELL_TYPE_KEY] == 'A549')]
pert_adata = adata[(adata.obs[PERT_KEY] == "IFNAR1") & (adata.obs[CELL_TYPE_KEY] == 'A549')]



In [ ]:
del batch


In [27]:
import gc
torch.cuda.empty_cache()
gc.collect()
batch_size = 10


In [32]:
#Random sample of 10000 cells

np_reconstructed_pert = np.zeros(pert_adata.X.shape)
for i in range(0, pert_adata.X.shape[0], batch_size):
    batch = pert_adata.X[i:i+batch_size]
    batch = torch.from_numpy(batch).float().to(device)
    recon, is_expressed, mu, logvar = model.forward(batch, mode='reconstruction')
    np_reconstructed_pert[i:i+batch_size] = recon.cpu().detach().numpy()

np_reconstructed_control = np.zeros(control_adata.X.shape)
for i in range(0, control_adata.X.shape[0], batch_size):
    batch = control_adata.X[i:i+batch_size]
    batch = torch.from_numpy(batch).float().to(device)
    recon, is_expressed, mu, logvar = model.forward(batch, mode='reconstruction')
    np_reconstructed_control[i:i+batch_size] = recon.cpu().detach().numpy()


adata_reconstructed_pert = sc.AnnData(X=np_reconstructed_pert,var=pert_adata.var)
adata_reconstructed_control = sc.AnnData(X=np_reconstructed_control,var=control_adata.var)


In [29]:
from os import listdir
from scipy.sparse import issparse
import anndata
import scanpy as sc
import numpy as np
import pandas as pd

from scipy.stats import pearsonr

import logging

logger = logging.getLogger(__name__)

def r2_mse_filename(pert, cell):
    return f'r2_and_mse_{pert}_{cell}.json'

def c_r_filename(pert, cell):
    return f'c_r_results_{pert}_{cell}.json'

def DEGs_overlap_filename(pert, cell):   
    return f'DEGs_overlaps_{pert}_{cell}.json'


def get_DEG_with_direction(gene, score):
    if score > 0:
        return(f'{gene}+')
    else:
        return(f'{gene}-')
        
def to_dense(X):
    if issparse(X):
        return X.toarray()
    else:
        return np.asarray(X)

def get_DEGs(control_adata, target_adata):
    temp_concat = anndata.concat([control_adata, target_adata], label = 'batch')
    sc.tl.rank_genes_groups(
        temp_concat, 'batch', method='wilcoxon', 
        groups = ['1'], ref = '0', rankby_abs = True, tie_correct=True
    )

    rankings = temp_concat.uns['rank_genes_groups']
    result_df = pd.DataFrame({'scores': rankings['scores']['1'],
                     'pvals_adj': rankings['pvals_adj']['1'],
                     'lfc': rankings['logfoldchanges']['1']},
                    index = rankings['names']['1'])
    return result_df

def get_eval(ctrl_adata, true_adata, pred_adata, DEGs, DEG_vals, pval_threshold, lfc_threshold):
        
    results_dict =  {}
    
    logger.debug(f"Computing R, R2, and MSE metrics")

    ctrl_mean = to_dense(ctrl_adata.X).mean(axis = 0)

    true_mean = to_dense(true_adata.X).mean(axis = 0)
    true_var = to_dense(true_adata.X).var(axis = 0)
    
    pred_mean = to_dense(pred_adata.X).mean(axis = 0)
    pred_var = to_dense(pred_adata.X).var(axis = 0)
    
    true_corr_mtx = np.corrcoef(to_dense(true_adata.X), rowvar=False).flatten()
    true_cov_mtx = np.cov(to_dense(true_adata.X), rowvar=False).flatten()
        
    pred_corr_mtx = np.corrcoef(to_dense(pred_adata.X), rowvar=False).flatten()
    pred_cov_mtx = np.cov(to_dense(pred_adata.X), rowvar=False).flatten()

    true_sub_diff = true_mean - ctrl_mean
    pred_sub_diff = pred_mean - ctrl_mean

    true_diff = true_mean/ctrl_mean
    pred_diff = pred_mean/ctrl_mean

    true_diff_mask = (np.isnan(true_diff) | np.isinf(true_diff))
    pred_diff_mask = (np.isnan(pred_diff) | np.isinf(pred_diff))
    
    common_mask = true_diff_mask | pred_diff_mask
    true_fold_diff = np.ma.array(true_diff, mask=common_mask).compressed()
    pred_fold_diff = np.ma.array(pred_diff, mask=common_mask).compressed()

    results_dict['all_genes_mean_sub_diff_R'] = pearsonr(true_sub_diff, pred_sub_diff)[0]
    results_dict['all_genes_mean_sub_diff_R2'] = pearsonr(true_sub_diff, pred_sub_diff)[0]**2
    results_dict['all_genes_mean_sub_diff_MSE'] = (np.square(true_sub_diff - pred_sub_diff)).mean(axis=0)

    results_dict['all_genes_mean_fold_diff_R'] = pearsonr(true_fold_diff, pred_fold_diff)[0]
    results_dict['all_genes_mean_fold_diff_R2'] = pearsonr(true_fold_diff, pred_fold_diff)[0]**2
    results_dict['all_genes_mean_fold_diff_MSE'] = (np.square(true_fold_diff - pred_fold_diff)).mean(axis=0)
    
    results_dict['all_genes_mean_R'] = pearsonr(true_mean, pred_mean)[0]
    results_dict['all_genes_mean_R2'] = pearsonr(true_mean, pred_mean)[0]**2
    results_dict['all_genes_mean_MSE'] = (np.square(true_mean - pred_mean)).mean(axis=0)

    results_dict['all_genes_var_R'] = pearsonr(true_var, pred_var)[0]
    results_dict['all_genes_var_R2'] = pearsonr(true_var, pred_var)[0]**2
    results_dict['all_genes_var_MSE'] = (np.square(true_var - pred_var)).mean(axis=0)
   
    corr_nas = np.logical_or(np.isnan(true_corr_mtx), np.isnan(pred_corr_mtx))
    cov_nas = np.logical_or(np.isnan(true_cov_mtx), np.isnan(pred_cov_mtx))

    results_dict['all_genes_corr_mtx_R'] = pearsonr(true_corr_mtx[~corr_nas], pred_corr_mtx[~corr_nas])[0]
    results_dict['all_genes_corr_mtx_R2'] = pearsonr(true_corr_mtx[~corr_nas], pred_corr_mtx[~corr_nas])[0]**2
    results_dict['all_genes_corr_mtx_MSE'] = (np.square(true_corr_mtx[~corr_nas] - pred_corr_mtx[~corr_nas])).mean(axis=0)

    results_dict['all_genes_cov_mtx_R'] = pearsonr(true_cov_mtx[~cov_nas], pred_cov_mtx[~cov_nas])[0]
    results_dict['all_genes_cov_mtx_R2'] = pearsonr(true_cov_mtx[~cov_nas], pred_cov_mtx[~cov_nas])[0]**2
    results_dict['all_genes_cov_mtx_MSE'] = (np.square(true_cov_mtx[~cov_nas] - pred_cov_mtx[~cov_nas])).mean(axis=0)

    if lfc_threshold:   
        significant_DEGs = DEGs[(DEGs['pvals_adj'] < pval_threshold) & (abs(DEGs) > lfc_threshold)]
    else:
        significant_DEGs = DEGs[DEGs['pvals_adj'] < pval_threshold]
    num_DEGs = len(significant_DEGs)
    DEG_vals.insert(0, num_DEGs)


    logger.debug(f"Significant DEGs {significant_DEGs}")
    
    for val in DEG_vals:

        logger.debug(f"Computing R, R2, and MSE metrics for top {val} DEGs")

        #If val == 1 we can't
        if ((val > num_DEGs) or (val == 0) or (val == 1)):
            results_dict[f'Top_{val}_DEGs_sub_diff_mean_R'] = None
            results_dict[f'Top_{val}_DEGs_sub_diff_mean_R2'] = None
            results_dict[f'Top_{val}_DEGs_sub_diff_mean_MSE'] = None
            
            results_dict[f'Top_{val}_DEGs_fold_diff_mean_R'] = None
            results_dict[f'Top_{val}_DEGs_fold_diff_mean_R2'] = None
            results_dict[f'Top_{val}_DEGs_fold_diff_mean_MSE'] = None
            
            results_dict[f'Top_{val}_DEGs_mean_R'] = None
            results_dict[f'Top_{val}_DEGs_mean_R2'] = None
            results_dict[f'Top_{val}_DEGs_mean_MSE'] = None

            results_dict[f'Top_{val}_DEGs_var_R'] = None
            results_dict[f'Top_{val}_DEGs_var_R2'] = None
            results_dict[f'Top_{val}_DEGs_var_MSE'] = None
            
            results_dict[f'Top_{val}_DEGs_corr_mtx_R'] = None
            results_dict[f'Top_{val}_DEGs_corr_mtx_R2'] = None
            results_dict[f'Top_{val}_DEGs_corr_mtx_MSE'] = None
            
            results_dict[f'Top_{val}_DEGs_cov_mtx_R'] = None
            results_dict[f'Top_{val}_DEGs_cov_mtx_R2'] = None
            results_dict[f'Top_{val}_DEGs_cov_mtx_MSE'] = None
        
        else:
            top_DEGs = significant_DEGs.iloc[0:val].index

            logger.debug(f"Top DEGs: {top_DEGs}")


            #Reshape --> If there is a single gene, the shape is (1,) and we need to reshape it to (1,1)

            ctrl_mean = to_dense(ctrl_adata[:,top_DEGs].X).mean(axis = 0)
            
            true_mean = to_dense(true_adata[:,top_DEGs].X).mean(axis = 0)

            logger.debug(f"Shape ctrl_adata with top DEGs: {ctrl_adata[:,top_DEGs].X.shape}, shape true_adata with top DEGs: {true_adata[:,top_DEGs].X.shape}")


            true_var = to_dense(true_adata[:,top_DEGs].X).var(axis = 0)
            true_corr_mtx = np.corrcoef(to_dense(true_adata[:,top_DEGs].X), rowvar=False).flatten()
            true_cov_mtx = np.cov(to_dense(true_adata[:,top_DEGs].X), rowvar=False).flatten()

            pred_mean = to_dense(pred_adata[:,top_DEGs].X).mean(axis = 0)
            logger.debug(f"Shape of true_mean shape: {true_mean.shape}, ctrl_mean shape: {ctrl_mean.shape}, pred_mean shape: {pred_mean.shape}")

            pred_var = to_dense(pred_adata[:,top_DEGs].X).var(axis = 0)
            pred_corr_mtx = np.corrcoef(to_dense(pred_adata[:,top_DEGs].X), rowvar=False).flatten()
            pred_cov_mtx = np.cov(to_dense(pred_adata[:,top_DEGs].X), rowvar=False).flatten()

            logger.debug(f"Shape of true_var shape: {true_var.shape}, pred_var shape: {pred_var.shape}")

            true_sub_diff = true_mean - ctrl_mean
            pred_sub_diff = pred_mean - ctrl_mean
        
            true_diff = true_mean/ctrl_mean
            pred_diff = pred_mean/ctrl_mean
        
            true_diff_mask = (np.isnan(true_diff) | np.isinf(true_diff))
            pred_diff_mask = (np.isnan(pred_diff) | np.isinf(pred_diff))
            
            common_mask = true_diff_mask | pred_diff_mask
            true_fold_diff = np.ma.array(true_diff, mask=common_mask).compressed()
            pred_fold_diff = np.ma.array(pred_diff, mask=common_mask).compressed()

            results_dict[f'Top_{val}_DEGs_sub_diff_R'] = pearsonr(true_sub_diff, pred_sub_diff)[0]
            results_dict[f'Top_{val}_DEGs_sub_diff_R2'] = pearsonr(true_sub_diff, pred_sub_diff)[0]**2
            results_dict[f'Top_{val}_DEGs_sub_diff_MSE'] = (np.square(true_sub_diff - pred_sub_diff)).mean(axis=0)
        
            results_dict[f'Top_{val}_DEGs_fold_diff_R'] = pearsonr(true_fold_diff, pred_fold_diff)[0]
            results_dict[f'Top_{val}_DEGs_fold_diff_R2'] = pearsonr(true_fold_diff, pred_fold_diff)[0]**2
            results_dict[f'Top_{val}_DEGs_fold_diff_MSE'] = (np.square(true_fold_diff - pred_fold_diff)).mean(axis=0)
    
            results_dict[f'Top_{val}_DEGs_mean_R'] = pearsonr(true_mean, pred_mean)[0]
            results_dict[f'Top_{val}_DEGs_mean_R2'] = pearsonr(true_mean, pred_mean)[0]**2
            results_dict[f'Top_{val}_DEGs_mean_MSE'] = (np.square(true_mean - pred_mean)).mean(axis=0)

            results_dict[f'Top_{val}_DEGs_var_R'] = pearsonr(true_var, pred_var)[0]
            results_dict[f'Top_{val}_DEGs_var_R2'] = pearsonr(true_var, pred_var)[0]**2
            results_dict[f'Top_{val}_DEGs_var_MSE'] = (np.square(true_var - pred_var)).mean(axis=0)
            
            corr_nas = np.logical_or(np.isnan(true_corr_mtx), np.isnan(pred_corr_mtx))
            cov_nas = np.logical_or(np.isnan(true_cov_mtx), np.isnan(pred_cov_mtx))

            results_dict[f'Top_{val}_DEGs_corr_mtx_R'] = pearsonr(true_corr_mtx[~corr_nas], pred_corr_mtx[~corr_nas])[0]
            results_dict[f'Top_{val}_DEGs_corr_mtx_R2'] = pearsonr(true_corr_mtx[~corr_nas], pred_corr_mtx[~corr_nas])[0]**2
            results_dict[f'Top_{val}_DEGs_corr_mtx_MSE'] = (np.square(true_corr_mtx[~corr_nas] - pred_corr_mtx[~corr_nas])).mean(axis=0)

            results_dict[f'Top_{val}_DEGs_cov_mtx_R'] = pearsonr(true_cov_mtx[~cov_nas], pred_cov_mtx[~cov_nas])[0]
            results_dict[f'Top_{val}_DEGs_cov_mtx_R2'] = pearsonr(true_cov_mtx[~cov_nas], pred_cov_mtx[~cov_nas])[0]**2
            results_dict[f'Top_{val}_DEGs_cov_mtx_MSE'] = (np.square(true_cov_mtx[~cov_nas] - pred_cov_mtx[~cov_nas])).mean(axis=0)

    return results_dict

def get_DEG_Coverage_Recall(true_DEGs, pred_DEGs, p_cutoff):
    sig_true_DEGs = true_DEGs[true_DEGs['pvals_adj'] < p_cutoff]
    true_DEGs_with_direction = [get_DEG_with_direction(gene,score) for gene, score in zip(sig_true_DEGs.index, sig_true_DEGs['scores'])]
    sig_pred_DEGs = pred_DEGs[pred_DEGs['pvals_adj'] < p_cutoff]
    pred_DEGs_with_direction = [get_DEG_with_direction(gene,score) for gene, score in zip(sig_pred_DEGs.index, sig_pred_DEGs['scores'])]
    num_true_DEGs = len(true_DEGs_with_direction)
    num_pred_DEGs = len(pred_DEGs_with_direction)
    num_overlapping_DEGs = len(set(true_DEGs_with_direction).intersection(set(pred_DEGs_with_direction)))
    if num_true_DEGs > 0: 
        COVERAGE = num_overlapping_DEGs/num_true_DEGs
    else:
        COVERAGE = None
    if num_pred_DEGs > 0:
        RECALL = num_overlapping_DEGs/num_pred_DEGs
    else:
        RECALL = None
    return COVERAGE, RECALL

def get_DEGs_overlaps(true_DEGs, pred_DEGs, DEG_vals, pval_threshold, lfc_threshold):
    if lfc_threshold:
        significant_true_DEGs = true_DEGs[(true_DEGs['pvals_adj'] < pval_threshold) & (abs(true_DEGs['lfc']) > lfc_threshold)]
        significant_pred_DEGs = pred_DEGs[(pred_DEGs['pvals_adj'] < pval_threshold) & (abs(pred_DEGs['lfc']) > lfc_threshold)]
    else:
        significant_true_DEGs = true_DEGs[true_DEGs['pvals_adj'] < pval_threshold]
        significant_pred_DEGs = pred_DEGs[pred_DEGs['pvals_adj'] < pval_threshold]

    true_DEGs_for_comparison = [get_DEG_with_direction(gene,score) for gene, score in zip(significant_true_DEGs.index, significant_true_DEGs['scores'])]   
    pred_DEGs_for_comparison = [get_DEG_with_direction(gene,score) for gene, score in zip(significant_pred_DEGs.index, significant_pred_DEGs['scores'])]
    
    logger.debug(f"Computing DEG overlaps, # of significant DEGs in true data: {len(true_DEGs_for_comparison)}, # of significant DEGs in pred data: {len(pred_DEGs_for_comparison)}")
    num_DEGs = len(significant_true_DEGs)
    DEG_vals.insert(0, num_DEGs)
    
    results = {}
    for val in DEG_vals:
        if val > num_DEGs:
            results[f'Overlap_in_top_{val}_DEGs'] = None
        else:
            results[f'Overlap_in_top_{val}_DEGs'] = len(set(true_DEGs_for_comparison[0:val]).intersection(set(pred_DEGs_for_comparison[0:val])))

    intersection = len(set(true_DEGs_for_comparison).intersection(set(pred_DEGs_for_comparison)))
    union = len(set(true_DEGs_for_comparison).union(set(pred_DEGs_for_comparison)))
    if union > 0:
        results['Jaccard'] = intersection/union
    else:
        results['Jaccard'] = None
    
    return results

In [34]:


degs_gt = get_DEGs(control_adata, pert_adata)

degs_gt_rec = get_DEGs(control_adata, adata_reconstructed_pert)

degs_rec_rec = get_DEGs(adata_reconstructed_control, adata_reconstructed_pert)


/orcd/archive/abugoot/001/Projects/opitcho/miniforge3/envs/sandbox/lib/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:358: RuntimeWarning: invalid value encountered in divide
  scores[group_index, :] = (
/orcd/archive/abugoot/001/Projects/opitcho/miniforge3/envs/sandbox/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [38]:

pdefault=0.05

significant_degs = degs_gt[degs_gt['pvals_adj'] < pdefault]
significant_degs_pred = degs_gt_rec[degs_gt_rec['pvals_adj'] < pdefault]

In [39]:
len(significant_degs.index[:10].intersection(significant_degs_pred.index[:10])), \
len(significant_degs.index[:25].intersection(significant_degs_pred.index[:25])), \
len(significant_degs.index[:50].intersection(significant_degs_pred.index[:50])), \
len(significant_degs.index[:100].intersection(significant_degs_pred.index[:100])), \
len(significant_degs.index.intersection(significant_degs_pred.index))

(0, 0, 0, 0, 508)